Analysis
#1. From a sample of 100 countries, it appears that they are randomly selected from all around the globe. 
#2. There is almost no correlation between cloudiness and latitude.
#3. The correlation between humidity and latitude is very weak.
#4. There is a relatively strong correlation bewteen latitude and temperature.

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)
 



## Generate Cities List

# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1900)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1900)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count

#print(cities)
len(cities)

#See what the data is and what I am working with. Basically I am getting the data here.  

response = requests.get('http://api.openweathermap.org/data/2.5/weather?q=portland&units=imperial&appid=e08650aa218de36f561d54df8fd78503').json()
print(json.dumps(response, indent=2, sort_keys=True))

#I am going to create a DataFrame for the criteria I am looking at
#criteria = City, Temperture, Humidity, Cloudiness, Wind Speed, Latitude, Longitude. I will use this data for my maps as well
weatherdf = pd.DataFrame({"City": cities, "Temperture": "", "Humidity": "",
                         "Cloudiness": "", "Wind Speed": "", "Latitude": "", "Longitude": "", "Date": "", "Country": ""})
weatherdf.head()

## Perform API Calls

# OpenWeatherMap API Key
api_key = api_keys.api_key

# Starting URL for Weather Map API Call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key + "&q="
print(url)


#Create blank columns in DataFrame for Data 
#City, Temperture, Humidity, Cloudiness, Wind Speed, Latitude, Longitude
temperture = []
humidity = []
cloudiness = []
windspeed = []
latitude = []
date = []
country = []
print(f"Start the API calls")
print(f"---------------")

#uptop but I want to make sure I have it down low



#Counter
#rowcounter = 0

#Loop through and grab the data to put in our weather DataFrame we created up top.
#
for index, row in weatherdf.iterrows():
    city = row["City"]
    cityurl = url + city
    
    response = requests.get(cityurl).json()
    print(f"City Name: ", {city[0]})
        
#City, Temperture, Humidity, Cloudiness, Wind Speed, Latitude, Longitude**Parsing
    try:
        print(f"Processing Record # {index+1} | {cityurl}")
        print(url)    
    
        weatherdf.loc[index, 'City'] = response['name']
        weatherdf.loc[index, 'Temperture'] = response['main']['temp']
        weatherdf.loc[index, 'Humidity'] = response['main']['humidity']
        weatherdf.loc[index, 'Cloudiness'] = response['clouds']['all']
        weatherdf.loc[index, 'Wind Speed'] = response['main']['temp']
        weatherdf.loc[index, 'Latitude'] = response['coord']['lat']
        weatherdf.loc[index, 'Longitude'] = response['coord']['lon']
        weatherdf.loc[index, 'Date'] = response['dt']
        weatherdf.loc[index, 'Country'] = response['sys']['country']
        
       
           
    
    except (KeyError):
        
        print(f"Information missing for ", {city[0]})
        print(f"-------------------------------------")
        pass 
            

            

#how many do I have
print(weatherdf['City'].nunique())



#Find and replace then drop empties
weatherdf['Temperture'].replace('', np.nan, inplace=True)
weatherdf.dropna(subset=['Temperture'], how='any', inplace=True)
print(weatherdf['City'].nunique())

#I just went down from 733 to 644. I did change the Random number because I felt it was too close to 500


#printing out my appended dataframe with updated info
weatherdf.head(100)

#looks like the random really selected countries around the globe. I just looked at the first 100 and it looks to be spread out

#save the dataframe as a csv
weatherdf.to_csv('output_data_file_cities', index=False)

#Plot Temperature (F) vs. Latitude

plt.scatter(weatherdf['Latitude'], weatherdf['Temperture'], marker='o', alpha= .5, facecolors="red", edgecolors= "pink", 
            label="City", linewidths=2)



plt.title("Latitude vs. Temperature (%s)" % time.strftime("%x"))
plt.xlabel("Latitude")
plt.ylabel("Temperature (F)")
plt.grid(color='grey', linewidth=.2)
#plt.savefig(output_data_file) got an error with this code
plt.savefig("TempvsLatitude.png", edgecolor='black')


plt.show()

#Plot Humidity (%) vs. Latitude
plt.scatter(weatherdf['Latitude'], weatherdf['Humidity'], marker='o', alpha= .5, facecolors="blue", edgecolors= "grey", 
            label="City", linewidths=2)



plt.title("Latitude vs. Humidity (%s)" % time.strftime("%x"))
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.grid(color='grey', linewidth=.2)

plt.savefig("HumidityvLatitude.png", edgecolor='black')


plt.show()

# Plot Cloudiness (%) vs. Latitude
plt.scatter(weatherdf['Latitude'], weatherdf['Cloudiness'], marker='o', alpha= .5, facecolors="yellow", edgecolors= "orange", 
            label="City", linewidths=2)



plt.title("Latitude vs. Cloudiness (%s)" % time.strftime("%x"))
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.grid(color='grey', linewidth=.2)

plt.savefig("CloudinessvLatitude.png", edgecolor='black')


plt.show()

# Plot Cloudiness (%) vs. Latitude
plt.scatter(weatherdf['Latitude'], weatherdf['Wind Speed'], marker='o', alpha= .5, facecolors="purple", edgecolors= "purple", 
            label="City", linewidths=2)



plt.title("City Latitude vs. Wind Speed (%s)" % time.strftime("%x"))
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.grid(color='grey', linewidth=.2)

plt.savefig("WindSpeedvLatitude.png", edgecolor='black')


plt.show()

